In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
import sys, os
sys.path.insert(0,'/global/homes/d/dgct/Repos/metatlas/')

#sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )

from metatlas.helpers import dill2plots as dp
from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas.helpers import rt_corrector as rt_corrector
from metatlas.helpers import chromatograms_mp_plots as cp
from metatlas.helpers import chromplotplus as cpp
from metatlas.helpers import spectralprocessing as sp
from metatlas.helpers import fastanalysis as fa
import metatlas.metatlas_objects as metob
from metatlas.helpers import mzmine_helpers as mzm

import qgrid

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

import time
import dill
import numpy as np
import multiprocessing as mp
import pandas as pd
import copy

import matplotlib.pyplot as plot

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning:


This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.




In [2]:
#default ema atlas: EMA_pos%50447%
atlases = metob.retrieve('Atlas',name='SK_EMA_MP-SolapP_POS_20170914-2108-proton',username='*')
# atlases = metob.retrieve('Atlas',name='SK-ZZ-MAIZE-HILIC-NEG-EMA50447',username='*')
names = []
for i,a in enumerate(atlases):
    print(i,a.name,pd.to_datetime(a.last_modified,unit='s'))#len(a.compound_identifications)

(0, u'SK_EMA_MP-SolapP_POS_20170914-2108-proton', Timestamp('2017-09-15 04:23:39'))


In [3]:
atlas = atlases[0]
print atlas.name
print atlas.username

SK_EMA_MP-SolapP_POS_20170914-2108-proton
smkosina


In [4]:
dp = reload(dp)
# group_names_to_remove = pd.read_csv('/global/homes/b/bpb/Downloads/pks_files_to_remove.csv')['group'].tolist()

groups = dp.select_groups_for_analysis(name = '%20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG','QC','Bl'])#['QC','Blank'])


4
(0, u'20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760_POS_M1', datetime.datetime(2017, 9, 14, 20, 58, 35))
(1, u'20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760_POS_L1', datetime.datetime(2017, 9, 14, 20, 58, 40))
(2, u'20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760_POS_C1', datetime.datetime(2017, 9, 14, 20, 58, 44))
(3, u'20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760_POS_R1', datetime.datetime(2017, 9, 14, 20, 58, 51))


In [8]:
output_dir = '/global/homes/d/dgct/fast_analysis/new_parameters_py/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
reload(fa)
fa.filter_and_dump(atlas, groups, output_dir,
                   min_intensity = 1e5,
                   rt_tolerance = .25,
                   mz_tolerance = 10,
                   min_msms_score = .7, allow_no_msms = False,
                   min_num_frag_matches = 3,  min_relative_frag_intensity = .05)